# Robusthedstest. Vurdering af effekt på udelukkende på data fra 2021.

###### Note: Koden her er identisk med den fra den primære analyse. Den eneste forskel er, at jeg kun kører den på data fra 2021.

Dette bilag vil redegøre for indsamlingen og analysen af robusthedstesten. 

Første af snit omhandler hvordan dataene er indsamlet. 

Herefter følges klargørelse og analyse.

###### Centralt er to strategier i min dataklargørelse: 
1. Sørge for, at der kan skabes et aggregeret overblik over ord på tværs af jobannoncer (med stadig mulighed for sektoradskillelse)
2. Sørge for, at hver jobannonce bevarer sin unikke kombination af ord


In [ ]:
# NOTE TO SELF: SKRIV LIGE DATAPROCESSEN REN - HVOR DU I STEDET FOR AT GØRE DET TO GANGE, GØR DET ÉN GANG - 
# OG DELER OFFENTLIGE/PRIVAT SENERE I PROCESSEN

In [2]:
# Dette er de forskellige udviklerpakker, som jeg bruger undervejs. Skal køres. 
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import nltk
import string
import lemmy
import numpy
import sys

## Her skabes mit datasæt bestående af jobannoncer fra Jobindex 

In [2]:
# import pandas as pd
# import time
# import requests
# from bs4 import BeautifulSoup

# index = []
# list_companies = []
# beskrivelse_liste = []
# antal_stjerner = []
# month_list = []
# year_list = []
# day_list = []
# url_liste = []
# category_job = []

# antal_sider = 1130

# for j in range(1, antal_sider + 1):
#     url = "https://www.jobindex.dk/jobsoegning/kontor/jura?jobage=archive&maxdate=20211121&mindate=20050101&page={}&q=jurist+advokat".format(j)
#     result = requests.get(url)
#     src = result.content
#     soup = BeautifulSoup(src, 'html.parser')
#     jobs = soup.find_all("div", {"class": 'PaidJob'})


#     # Kodning
    
#     for i, job in enumerate(jobs):
        
#         # Navnet på organisationen
#         try:
#             company = job.p.a.b.text
#             list_companies.append(company)
#         except:
#             list_companies.append('NaN')

#         # tekst fra kroppen af annoncen
#         text = ''
#         for p in job.find_all('p')[1:]:
#             text += p.text

#          # bulletpoints fra annoncen
#         opgaver_liste = ''
#         opgaver = job.find_all('li')


#         for opgave in opgaver:
#             if '\n' not in opgave.text:
                
#                 text += opgave.text

#         beskrivelse_liste.append(text)

#         try: 
#             stjerner = job.find('span', {'class': 'sr-only'}).text.split()[0]
#             antal_stjerner.append(stjerner)
#         except:
#             antal_stjerner.append('NaN')

#         # data
#         dato = str(job.find('time'))
#         year = dato[16:20]
#         month = dato[21:23]
#         day = dato[24:26]

#         month_list.append(month)
#         year_list.append(year)
#         day_list.append(day)

#         link = job.find_all('a')[6]['href']
#         url_liste.append(link)



        
#     time.sleep(2)
    


In [3]:
df = pd.DataFrame()

In [4]:
df['index'] = pd.Series(index)
df['company'] = pd.Series(list_companies)
df['jobbeskrivelse'] = pd.Series(beskrivelse_liste)
df['antal_stjerner'] = pd.Series(antal_stjerner)
df['month'] = pd.Series(month_list)
df['year'] = pd.Series(year_list)
df['day'] = pd.Series(day_list)
df['url'] = pd.Series(url_liste)

<ipython-input-4-68d20d33f58d>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['index'] = pd.Series(index)


In [5]:
df["company"].iloc[10000]

'BankInvest Gruppen'

In [6]:
df["jobbeskrivelse"].iloc[10000]

'\nVi søger en erfaren jurist til selvstændigt at give juridisk rådgivning til BankInvest Gruppens direktion og afdelinger og til at løse en række opgaver under overskriften ”legal risk management”, herunder arbejde med implementering af ny lovgivning. Du vil i vid udstrækning selv være med til at identificere relevante arbejdsopgaver. \nDu vil indgå som en del af Legal Department, som ud over en chefjurist består af tre jurister/advokater samt to sekretærer. '

In [7]:
df["year"].iloc[10000]

'2008'

In [324]:
df.to_pickle("final_scraping.pkl") # gemmer ovenstående dataframe til en pickle-fil

## Her klargøres og opryddes datasættet

In [4]:
df = pd.read_pickle("final_scraping.pkl") # indlæser pickle-fil

df["company"]

0                      Telenor
1                   Alm. Brand
2               Herlev Kommune
3              Skattestyrelsen
4                       PenSam
                 ...          
10418            SkandiaBanken
10419       Realkredit Danmark
10420                BRFkredit
10421    Dansk Retursystem A/S
10422              Banedanmark
Name: company, Length: 10359, dtype: object

#### Her frasorteres udenlandske annoncer:

In [5]:
#Frasorter de engelske jobannoncer
df['english'] = [0 if 'æ' in x or 'ø' in x or 'å' in x else 2 for x in df['jobbeskrivelse']]

df = df[df.english != 2]


#### Her kategoriseres offentlige jobannoncer, baseret på navnet på afsendervirksomheden:

In [43]:
# Sorterer offentlige og private jobannoncer fra hinanden
# offentlig er 1 og privat 0

df['offentlig'] = [1 if 'Kommune' in x # først statslige myndigheder
                   or 'kommune' in x
                   or 'ministerie' in x 
                   or 'Ministerie' in x
                   or 'minister' in x
                   or 'styrelse' in x 
                   or 'Styrelse' in x 
                   or 'Region' in x
                   or 'Staten' in x
                   or 'myndighed' in x 
                   or 'Myndighed' in x
                   or 'tilsyn' in x 
                   or 'Politi' in x 
                   or 'politi' in x 
                   or 'Forsvaret'in x
                   or 'DR' in x
                   or 'Banedanmark' in x
                   or 'Retten' in x
                   or 'Grønland' in x
                   or 'Selvstyre' in x
                   or 'Kriminalforsorgen'in x
                   or 'Statens IT' in x
                   or 'Danmarks Statistik' in x
                   or 'Sundhed.dk' in x
                   or 'Bibliotek' in x
                   or 'SKAT' in x
                   or 'Jobcenter' in x
                   or 'Hjemmeværnskommandoen' in x
                   or 'Statsforvaltningen' in x
                   or 'Nationalbank' in x
                   or 'Vejdirektoratet' in x
                   # under beskæftigelsesministeriet
                   or 'ATP' in x
                   or 'Arbejdsmarkedets Erhvervssikring' in x
                   # herefter uddannelsesinstitutioner
                   or 'Profession' in x
                   or 'Universitet'in x
                   or 'Copenhagen Business School' in x
                   or 'University College' in x
                   or 'Erhvervsskolen Nordsjælland' in x
                   or 'Erhvervsakademi' in x
                   or 'UC Syd' in x
                   or 'College' in x 
                   or 'Hotel- og Restaurantskolen' in x
                   or 'Kongelige' in x
                   or 'SIMAC' in x
                   # herefter råd og nævn
                   or 'Det Etiske Råd' in x
                   or 'Patienterstatningen' in x
                   or 'Nævn' in x
                   or 'nævn' in x
                   or 'Den Nationale Klageinstans mod Mobning' in x
                   or 'Rigsadvokaten' in x
                   or 'Rigsombudsmanden i Grønland' in x
                   or 'Den Europæiske Union' in x
                   or 'Nævnenes Hus' in x
                   or 'Dansk Varefakta Nævn' in x
                   # herefter statslige selskaber med fuld aktiemajoritet
                   or 'DSB' in x
                   or 'Vækstfonden' in x
                   or 'EKF Danmarks Eksportkredit' in x
                   or 'Finansiel Stabilitet' in x
                   or 'Investeringsfonden for udviklingslande (IFU)' in x
                   or 'Naviair' in x
                   or 'Evida' in x
                   or 'Sund & Bælt Holding A/S' in x
                   or 'Eksport Kredit Fonden' in x
                   or 'Nationalt Genom Center' in x
                   or 'Københavns Tekniske Skole' in x
                   or 'Energinet' in x
                   or "TV 2" in x
                   or "Nordsøenheden" in x
                   or "Bornholmstrafikken Holding A/S" in x
                   or "Danpilot" in x
                   or "Statens Ejendomssalg A/S" in x
                   or "Det Danske Klasselotteri A/S" in x
                   
                   
                   
                   else 0

for x in df['company']]

# Jeg bliver dog nød til at manuelt korregere fejl, som fx dette firma der er blevet klassificeret som offentlig grundet søgning efter "DR".

df.loc[df.company == "DRACHMANN ADVOKATER", 'offentlig'] = 0
df.loc[df.company == "DRACHMANN EJENDOMSADMINISTRATION A/S", 'offentlig'] = 0

# df.loc[df['offentlig'] == 1][500:550].head(30)



#### Her vurderes fordelingen af jobbeskrivelser på år. Jeg laver desuden et alternativt datasæt kun med observationer fra 2021.

In [44]:
# finder fordelingen af annoncer baseret på årstal

print(len(df[df['year'].str.contains("2021")]))

# alternativt datasæt for at måle robusthed (fjerner år 2007-2020)
df = df[~df.year.str.contains("2007|2008|2009|2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020")]

# fordelingen af offentlige og private annoncer i 2021

print(df[df['year'].str.contains("2021")].offentlig.value_counts())

1540
1    872
0    668
Name: offentlig, dtype: int64


#### Fordelinger af virksomheder baseret på sektortype:

In [46]:
# Hvor mange virksomheder fremfår fra det private og fra det offentlige?
# HUSK PRIVAT ER 0 og OFFENTLIG ER 1

df['offentlig'].value_counts()

1    872
0    668
Name: offentlig, dtype: int64

In [66]:
# vurder, om du skal fjerne nogle år...

In [106]:
print("There are {} observations and {} features in this dataset. \n".format(df.shape[0],df.shape[1]))

There are 1540 observations and 10 features in this dataset. 



In [10]:
# opdeling af datasæt i privat og offentlig karakter
offentlig_data = df.loc[df['offentlig'] == 1]
privat_data = df.loc[df['offentlig'] == 0]

# fokus på de pågældende jobbeskrivelser (overvej, om du stadigvæk vil bibeholde firmanavnet her?)
offentlig_text = offentlig_data.jobbeskrivelse
privat_text = privat_data.jobbeskrivelse

# lav en stor, som ikke adskiller de i sektorer (endnu)
all_text = df.jobbeskrivelse

#### Fordelingen af unikke virksomheder på sektor: 

In [47]:
# Skaber en liste over alle virksomheder i datasættet - delt op i privat og offentlig
# numpy.set_printoptions(threshold=sys.maxsize)

# df['company'].unique().tolist()

off_virk = offentlig_data['company'].unique().tolist()

priv_virk = privat_data['company'].unique().tolist()


#### (1) Her gøres min data til to forskellige sammenhængende lister af ord for hhv. den offentlige og private sektor. 

In [13]:
# skaber den sammenhængende text 
# husk at få fjernet \n

offentlig_sammenhæng_text = " ".join(job for job in offentlig_data.jobbeskrivelse)
privat_sammenhæng_text = " ".join(job for job in privat_data.jobbeskrivelse)

# gør alt tekst til små bogstaver

privat_små = \
     privat_sammenhæng_text.lower()
     #.replace(".","").replace(",","").replace("\n", " ").replace("-")

offentlig_små = offentlig_sammenhæng_text.lower()
    #.replace(".","").replace(",","").replace("\n", " ")

# fjerner tegnsætning og tal

offentlig_u_tegnsætning = offentlig_små.translate(str.maketrans('', '', string.punctuation))
offentlig_clean = offentlig_u_tegnsætning.translate(str.maketrans('', '', string.digits))

privat_u_tegnsætning = privat_små.translate(str.maketrans('', '', string.punctuation))
privat_clean = privat_u_tegnsætning.translate(str.maketrans('', '', string.digits))



In [14]:
# fjerner stopwords
stop_words  =\
    requests\
        .get("https://gist.githubusercontent.com/berteltorp/0cf8a0c7afea7f25ed754f24cfc2467b/raw/305d8e3930cc419e909d49d4b489c9773f75b2d6/stopord.txt")\
        .text \
        .split()


# manuel liste af stopwords
stop_words_manual = ["–", "mv", "”", "’", "bla","'","-"]

stop_words.extend(stop_words_manual)



In [30]:
# hvor mange ord er der i alt?
# print ("There are {} words in the combination of all review.".format(len(privat_tokens)))
# print ("There are {} words in the combination of all review.".format(len(offentlig_tokens)))

In [15]:
# her skabes min lemmatizer
lemmatize =\
    lambda word: lemmy\
        .load("da")\
        .lemmatize("", word)[0]

In [16]:
# FÆRDIG MED AT LAVE TO FORSKELLIGE ORD-LISTER UDEN SKELNING TIL UNIKKE JOBANNONCER 

#Her laver du Tokenization og Lemmatization

# Tokenization (og stop words)

privat_tokens = [word for word in nltk.word_tokenize(privat_clean) if word not in stop_words]

offentlig_tokens = [word for word in nltk.word_tokenize(offentlig_clean) if word not in stop_words]

# Lemmatization

privat_done_fluid = [lemmatize(word) for word in privat_tokens]

offentlig_done_fluid = [lemmatize(word) for word in offentlig_tokens]


#### (2) Herunder skaber jeg et datasæt hvori de unikke jobannoncer kan identificeres. Dette gør, at jeg fx kan finde en procentandel af brugen af PSM:

In [48]:
# Her skabes en analyerbar version af hele datamaterialet, samlet. 

#gjort små og fjerner \n
text_små = \
     all_text.str.lower().str.replace("\n", " ")

# # fjerner tegnsætning og tal
text_u_tegnsætning = text_små.str.translate(str.maketrans('', '', string.punctuation))
text_clean = text_u_tegnsætning.str.translate(str.maketrans('', '', string.digits))

# # fjerner stop words
text_u_stop = text_clean.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

# # Tokenization
text_tokens = text_u_stop.apply(nltk.word_tokenize)

# # Lemmatization 
text_done = text_tokens.apply(lambda lst:[lemmatize(word) for word in lst])

df_done = pd.concat([df.company, text_done, df.offentlig], axis=1)
           

In [49]:
# deler vores df_done op i hhv. private og offentlige beskrivelser

privat_done = df_done[df_done["offentlig"]==0]
offentlig_done = df_done[df_done["offentlig"]==1]


In [50]:
# den fulde df uden ovenstående sample
# df_done_final = df_done[~df_done.index.isin(df_sample.index)] OBS - DETTE ER GJORT FOROVEN

samlet_antal_off = len(df.loc[df['offentlig'] == 1])
samlet_antal_priv = len(df.loc[df['offentlig'] == 0])

# husk - det endelige datasæt på 10255?
samlet_antal_priv

668

### Søge sekventielt efter forekomsten af ord (PSM-indikatorer): 

In [20]:
# Vi vil gerne kunne se alle ord i cellerne
pd.set_option('display.max_colwidth', None)


In [41]:
# søge efter et ord

# df_done[df_done['jobbeskrivelse'].astype(str).str.contains("løn")]

#### Attraction to public policy making:

In [51]:
# Attraction to public policy making
# fortolke dele

#overvej om denne dimemsion er lidt for bredt defineret i din ordbog

verber_adjektiver_atp = "skabe|fortolke|dele|tilhøre|tilfredsstille|gavne|involvere|interessere|politisk|attraktiv|bryde|lide|høj"
substantiver_atp = "lov|land|fællesskab|holdning|program|gavn|menneske|politik|politisk|spil|tanke"


#Den første søgning efter verber og adjektiver
verber_adjektiver_atp_off = offentlig_done[offentlig_done['jobbeskrivelse'].astype(str).str.contains(verber_adjektiver_atp)]
verber_adjektiver_atp_priv = privat_done[privat_done['jobbeskrivelse'].astype(str).str.contains(verber_adjektiver_atp)]

#sekventiel søgning efter substantiver
alle_ordklasser_atp_off = verber_adjektiver_atp_off[verber_adjektiver_atp_off['jobbeskrivelse'].astype(str).str.contains(substantiver_atp)]
alle_ordklasser_atp_priv = verber_adjektiver_atp_priv[verber_adjektiver_atp_priv['jobbeskrivelse'].astype(str).str.contains(substantiver_atp)]

#udregner den offentlige andel
quotient = len(alle_ordklasser_atp_off) / samlet_antal_off 

percent = quotient * 100

print("Den offentlige procentandel:")
print(percent)

#udregner den private andel
quotient = len(alle_ordklasser_atp_priv) / samlet_antal_priv

percent = quotient * 100

print("Den private procentandel:")
print(percent)

# her indsættes attraction to public policy i df'en som binær-variabel
atp_index = [i for i in alle_ordklasser_atp_off.index]
atp_index.extend([i for i in alle_ordklasser_atp_priv.index])

df_done['ATP'] = df_done.index.map(lambda x : 1 if x in atp_index else 0)


Den offentlige procentandel:
41.51376146788991
Den private procentandel:
24.550898203592812


#### Commitment to the public interest:

In [52]:
# commitment to public interest

verber_adjektiver_cpi = "bidrage|gavne|tjene|god|meningsfuld|vigtig|uselvisk|fælles|hel"

substantiver_cpi = "borger|pligt|samfund|dyd|interesse|ydelse|ansat"

# Den første søgning efter verber og adjektiver blandt hhv. offentlig og private jobbeskrivelser
verber_adjektiver_cpi_off = offentlig_done[offentlig_done['jobbeskrivelse'].astype(str).str.contains(verber_adjektiver_cpi)]
verber_adjektiver_cpi_priv = privat_done[privat_done['jobbeskrivelse'].astype(str).str.contains(verber_adjektiver_cpi)]

# sekventiel søgning efter substantiver i det netop fremsøgte materiale
alle_ordklasser_cpi_off = verber_adjektiver_cpi_off[verber_adjektiver_cpi_off['jobbeskrivelse'].astype(str).str.contains(substantiver_cpi)]
alle_ordklasser_cpi_priv = verber_adjektiver_cpi_priv[verber_adjektiver_cpi_priv['jobbeskrivelse'].astype(str).str.contains(substantiver_cpi)]

#udregner den offentlige andel
quotient = len(alle_ordklasser_cpi_off) / samlet_antal_off 

percent = quotient * 100

print("Den offentlige procentandel:")
print(percent)

#udregner den private andel
quotient = len(alle_ordklasser_cpi_priv) / samlet_antal_priv

percent = quotient * 100

print("Den private procentandel:")
print(percent)

# her indsættes commitment to public interest i datasættet som binær variabel
cpi_index = [i for i in alle_ordklasser_cpi_off.index]
cpi_index.extend([i for i in alle_ordklasser_cpi_priv.index])

df_done['CPI'] = df_done.index.map(lambda x : 1 if x in cpi_index else 0)

# df_done[df_done['CPI']==1]

Den offentlige procentandel:
23.394495412844037
Den private procentandel:
12.125748502994012


#### Compassion: 

In [53]:
# compassion


verber_adjektiver_com = "blive|føle|behandle|følelsesmæssig|berørt|følsom|afhængig|retfærdig|mindre"
substantiver_com = "nød|sympati|afhængig|menneske|hensyn|velfærd|problem|fremmed|medfølelse|privilegie"

# Den første søgning efter verber og adjektiver blandt hhv. offentlig og private jobbeskrivelser
verber_adjektiver_com_off = offentlig_done[offentlig_done['jobbeskrivelse'].astype(str).str.contains(verber_adjektiver_com)]
verber_adjektiver_com_priv = privat_done[privat_done['jobbeskrivelse'].astype(str).str.contains(verber_adjektiver_com)]

# sekventiel søgning efter substantiver i det netop fremsøgte materiale
alle_ordklasser_com_off = verber_adjektiver_com_off[verber_adjektiver_com_off['jobbeskrivelse'].astype(str).str.contains(substantiver_com)]
alle_ordklasser_com_priv = verber_adjektiver_com_priv[verber_adjektiver_com_priv['jobbeskrivelse'].astype(str).str.contains(substantiver_com)]

#udregner den offentlige andel
quotient = len(alle_ordklasser_com_off) / samlet_antal_off 

percent = quotient * 100

print("Den offentlige procentandel:")
print(percent)

#udregner den private andel
quotient = len(alle_ordklasser_com_priv) / samlet_antal_priv

percent = quotient * 100

print("Den private procentandel:")
print(percent)

# her indsættes commitment to public interest i datasættet som binær variabel
com_index = [i for i in alle_ordklasser_com_off.index]
com_index.extend([i for i in alle_ordklasser_com_priv.index])

df_done['COM'] = df_done.index.map(lambda x : 1 if x in com_index else 0)

# df_done[df_done['COM']==1]

Den offentlige procentandel:
6.536697247706422
Den private procentandel:
2.245508982035928


#### Self-sacrifice:

In [54]:
# self-sacrifice

verber_adjektiver_ss = "gøre|give|yde|risikere|hjælpe|tilsidesætte|tjene|ofre|bidrage|god|personlig|uselvisk"
substantiver_ss = "forskel|pligt|gerning|samfund|skyld|tab|behov|borger|pligt|afsavn|hensyn|følelse"

# Den første søgning efter verber og adjektiver blandt hhv. offentlig og private jobbeskrivelser
verber_adjektiver_ss_off = offentlig_done[offentlig_done['jobbeskrivelse'].astype(str).str.contains(verber_adjektiver_ss)]
verber_adjektiver_ss_priv = privat_done[privat_done['jobbeskrivelse'].astype(str).str.contains(verber_adjektiver_ss)]

# sekventiel søgning efter substantiver i det netop fremsøgte materiale
alle_ordklasser_ss_off = verber_adjektiver_ss_off[verber_adjektiver_ss_off['jobbeskrivelse'].astype(str).str.contains(substantiver_ss)]
alle_ordklasser_ss_priv = verber_adjektiver_ss_priv[verber_adjektiver_ss_priv['jobbeskrivelse'].astype(str).str.contains(substantiver_ss)]

# udregner den offentlige andel
quotient = len(alle_ordklasser_ss_off) / samlet_antal_off 

percent = quotient * 100

print("Den offentlige procentandel:")
print(percent)

#udregner den private andel
quotient = len(alle_ordklasser_ss_priv) / samlet_antal_priv

percent = quotient * 100

print("Den private procentandel:")
print(percent)

# her indsættes commitment to public interest i datasættet som binær variabel
ss_index = [i for i in alle_ordklasser_ss_off.index]
ss_index.extend([i for i in alle_ordklasser_ss_priv.index])

df_done['SS'] = df_done.index.map(lambda x : 1 if x in ss_index else 0)

# df_done[df_done['SS']==1]


Den offentlige procentandel:
33.142201834862384
Den private procentandel:
27.694610778443113


### Forsøg på at finde ekstrinsiske indikatorer:

##### Ydre motivation: materiel

In [56]:
ydre_mot_verber_adj = "tilbyde|attraktiv|høj|konkurrencedygtig|stor|god"
ydre_mot_sub = "løn|personalegode|bonus"

# offentlig_done[offentlig_done['jobbeskrivelse'].astype(str).str.contains("løn|personalegode|bonus")]
# privat_done[privat_done['jobbeskrivelse'].astype(str).str.contains("tillæg|vederlag")]

In [57]:
# først søger jeg efter verber og adjektiver

priv_mot_verber_adj = privat_done[privat_done['jobbeskrivelse'].astype(str).str.contains(ydre_mot_verber_adj)]
off_mot_verber_adj = offentlig_done[offentlig_done['jobbeskrivelse'].astype(str).str.contains(ydre_mot_verber_adj)]

# herefter søger jeg efter substantiver

priv_ydre_mot_mate = priv_mot_verber_adj[priv_mot_verber_adj['jobbeskrivelse'].astype(str).str.contains(ydre_mot_sub)]
off_ydre_mot_mate = off_mot_verber_adj[off_mot_verber_adj['jobbeskrivelse'].astype(str).str.contains(ydre_mot_sub)]


# udregner den offentlige andel
quotient = len(off_ydre_mot_mate) / samlet_antal_off 

percent = quotient * 100

print("Den offentlige procentandel:")
print(percent)

#udregner den private andel
quotient = len(priv_ydre_mot_mate) / samlet_antal_priv

percent = quotient * 100

print("Den private procentandel:")
print(percent)


# her indsættes monetært incitament i datasættet som binær variabel
mi_index = [i for i in off_ydre_mot_mate.index]
mi_index.extend([i for i in priv_ydre_mot_mate.index])

df_done['MI'] = df_done.index.map(lambda x : 1 if x in mi_index else 0)



Den offentlige procentandel:
1.6055045871559634
Den private procentandel:
4.790419161676647


Okay, vi kan se, at der er forskel fra brug af hele datasættet også brug af færre år. Centralt er det dog, at den overordnede tendens er den samme - og derfor spiller det ikke en afgørende rolle, hvor meget data der medtages. 

### Signifikanstest af ovenstående forskel:

In [35]:
# t-test - hvis hver "psm-indikator" markeres med en kolonne, med enten 0 eller 1

from scipy.stats import ttest_ind

off = df_done[df_done['offentlig']==1]
priv = df_done[df_done['offentlig']==0]

In [36]:
# attraction to public policy, t-test

ttest_atp = ttest_ind(off['ATP'], priv['ATP']).pvalue

output_atp = f"{ttest_atp:.50f}"
print(output_atp)


0.00000000000244414945662624129420654287468290806300


In [37]:
# commitment to public interest, t-test

ttest_cpi = ttest_ind(off['CPI'], priv['CPI']).pvalue

output_cpi = f"{ttest_cpi:.50f}"

print(output_cpi)

0.00000001442477080029879734720646601445551149289770


In [38]:
# compassion, t-test

ttest_com = ttest_ind(off['COM'], priv['COM']).pvalue

output_com = f"{ttest_com:.50f}"

print(output_com)


0.00007491048103203798631290200038534976556547917426


In [39]:
# self-sacrifice, t-test

ttest_ss = ttest_ind(off['SS'], priv['SS']).pvalue

output_ss = f"{ttest_ss:.50f}"

print(output_ss)

0.02170888361558128559769542675894626881927251815796


In [40]:
# monetært incitament, t-test

ttest_mi = ttest_ind(off['MI'], priv['MI']).pvalue

output_mi = f"{ttest_mi:.50f}"

print(output_mi)

0.00026881765164920033749432204395191092771710827947
